In [1]:
import random
import bisect
import math
import csv
import os
import time
import ipaddress
import sympy

from pympler import asizeof

In [2]:
# Random Hash Function from Universal Hash family
# m here is the range of values that can be hashed into
class UniversalHash:
    def __init__(self, n, m, seed = 114514):
        # random.seed(seed) For getting deterministic output for analyzing
        self.p = sympy.nextprime(n)
        self.a = random.randint(1, self.p-1)
        self.b = random.randint(0, self.p-1)
        self.m = m
        self.hash_table = {} # Collision Counter
        self.collisions = 0
    
    # Collision Counter
    def detect_collisions(self, keys):
        for key in keys:
            hash_value = self.hash(key)
            if hash_value in self.hash_table:
                if key not in self.hash_table[hash_value]:
                    self.hash_table[hash_value].add(key)
                    self.collisions += 1
            else:
                self.hash_table[hash_value] = {key}

        return self.collisions
    
    def reset_collisions():
        self.hash_table = {}
        self.collisions = 0

    def hash(self, x):
        return (pow(self.a, x, self.p) + self.b) % self.p % self.m


In [3]:
def init(m, n, epsilon, phi, delta):
    l = int(6 * math.log2( 6 / delta) / epsilon**2)
    hash_function_size = math.ceil(4 * l**2 / delta)
    h = UniversalHash(n, hash_function_size)
    p = 6*l/m
    
    T1 = [None] * math.ceil(1/epsilon)
    T2 = [None] * math.ceil(1/phi)
    
    print("L: ",l,"H_size: ",hash_function_size, "T1_size: ", math.ceil(1/epsilon), "T2_size: ",math.ceil(1/phi), "P: ", p)

    return l, h, T1, T2, p

#@profile
def insert(x, m, l, h, T1, T2, phi, p):
    if random.random() < 6 * l / m:
        
        T1, T2 = misra_gries_update(x, h, T1, T2)
        
        return 1
    else:
        return 0
            
#@profile                
def misra_gries_update(x, h, T1, T2):
    isbreak = False
    h_x = h.hash(x)
    for i, item in enumerate(T1):
        if item and item[0] == h_x:
            item[1] += 1
            break
    else:
        for i, item in enumerate(T1):
            if item is None:
                T1[i] = [h_x, 1]
                if i < len(T2):
                    T2[i] = x
                isbreak = True
                break
            if item[1] == 0:
                item[0] = h_x
                item[1] = 1
                if i < len(T2):
                    T2[i] = x
                isbreak = True
                break
        if isbreak == False:
            for i, item in enumerate(T1):
                item[1] -= 1

    return bubble_Sort(T1, T2, x)

def report(T1, T2):
    result = [[t2_item[0], item[1]] for item, t2_item in zip(T1, T2) if item and t2_item]    
    print(result)
    return result

def bubble_Sort(T1, T2, x): # Can improve speed using qksort
    
    for i, itemi in enumerate(T1):
        for j, itemj in enumerate(T1[:-i-1]):
            if itemi and itemj:
                if T1[j] and T1[j+1] and T1[j][1] < T1[j+1][1]:
                    T1[j], T1[j+1] = T1[j+1], T1[j]
                    if j + 1 < len(T2):
                        T2[j], T2[j+1] = T2[j+1], T2[j]
                    elif j < len(T2):
                        T2[j] = x
    return T1,T2

In [4]:
cwd = os.getcwd()
file_path = os.path.join(cwd, 'V6.csv')           # Change stream here

with open(file_path, 'r') as file:
    row_count = sum(1 for _ in file)

m = row_count-1
n = 2**128
epsilon = 0.007
phi = 0.08
delta = 0.1
r_num = 0                                         # Set which colunm to read from

l, h, T1, T2, p = init(m, n, epsilon, phi, delta)

if epsilon >= phi:
    print("Warning: Epsilon Larger than Phi")
if p >= 1:
    print("Warning: p Larger than 1")

L:  723292 H_size:  20926052690560 T1_size:  143 T2_size:  13 P:  0.542469


In [5]:
print(m)

8000000


In [6]:
memory_h = asizeof.asizeof(h)
print(f"Memory usage: pum {memory_h} bytes")

memory_t1 = asizeof.asizeof(T1)
print(f"Memory usage: pum {memory_t1} bytes")

memory_t2 = asizeof.asizeof(T2)
print(f"Memory usage: pum {memory_t2} bytes")

Memory usage: pum 976 bytes
Memory usage: pum 1216 bytes
Memory usage: pum 176 bytes


In [7]:
start_time = time.time()
with open(file_path, 'r') as file:
    counter = 0
    inserted = 0
    reader = csv.reader(file)
    for row in reader:
        ip_address_str = row[r_num]                    # Change column here
        if counter == 0:
            print(ip_address_str)
        if counter > 0:
            
            integer_ip_address = int(ipaddress.IPv6Address(ip_address_str))
            inserted += insert(integer_ip_address, m, l, h, T1, T2, phi, p)
            #print("IP: ", ip_address)
            #print("Binary: ", binary_ip_address)
            #print("Integer: ", int.from_bytes(binary_ip_address, byteorder='big'))
            #print(h.hash(integer_ip_address))
            
        #if counter == 10:
        #    break
        counter += 1

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time, " seconds")

IPv6_Address
Elapsed time:  4284.856762170792  seconds


In [10]:
memory_h = asizeof.asizeof(h)
print(f"Memory usage: h {memory_h} bytes")

memory_t1 = asizeof.asizeof(T1)
print(f"Memory usage: T1 {memory_t1} bytes")

memory_t2 = asizeof.asizeof(T2)
print(f"Memory usage: T2 {memory_t2} bytes")

Memory usage: h 976 bytes
Memory usage: T1 16296 bytes
Memory usage: T2 784 bytes


In [12]:
print(T1)

[[13453536664911, 1072887], [13999430319909, 638624], [18927762829121, 378539], [15123246910217, 291404], [10602779082133, 204710], [4653462848078, 1], [14168047160913, 1], [6379151370490, 1], [15186728241682, 1], [9719184869947, 1], [1269180790769, 1], [15773027165501, 1], [10120439798235, 1], [16562804181550, 1], [10977160330325, 1], [18546580843196, 1], [9502242679263, 1], [4708376998940, 1], [7721976153759, 1], [14970744800386, 1], [13361817042786, 1], [16332136501017, 1], [1630468829210, 1], [20704077792831, 1], [3915996426701, 1], [12134391507013, 1], [701410618755, 1], [7860082604581, 1], [491734801341, 1], [5010241477156, 1], [9078531408565, 1], [2110845413474, 1], [13039948980708, 1], [20769174739142, 1], [1424521495357, 1], [9613927053915, 1], [20316735024967, 1], [11744250613303, 1], [1420865394292, 1], [11548751060709, 1], [10067636710726, 1], [16994436821595, 1], [2572549180352, 1], [6023615326622, 1], [12887558523643, 1], [15677291604832, 1], [7289727443087, 1], [10991977

In [13]:
print(T2)

[130899661903606533101852867240432767587, 110384156250175643175325737331030627956, 323793698876646609969570568750541558985, 75260133591338382011887897734966276291, 221941218907266329723269283006097611997, 46187186985849077124578569593943596837, 318648543685397671023067847623684177189, 200205756516636762614513633608205208778, 163728283234096744486017519831949203367, 39647661626287387087126745519830920236, 288146440203187210421451095566896202419, 196251790839559627631152828667005145482, 333511469534776090935640508623091892155]


In [11]:
#for i, item in enumerate(T2):
#    if item is not None:
#        print(str(ipaddress.IPv6Address(item)))
#        print(T1[i][1])

In [11]:
for i, item in enumerate(T2):
    if item is not None and T1[i][1]>= (phi-epsilon)*(6*l):
        temp_freq = T1[i][1]/(6*l)
        #print(temp_freq)
        if temp_freq > phi-epsilon:
            print(str(ipaddress.IPv6Address(item)))
            print(temp_freq)

627a:5b9c:84b9:7904:55b6:1984:ce24:1263
0.24722311321015578
530b:3716:bec1:682a:ca74:d1b8:aa66:1674
0.14715679605654886
f398:65e6:693b:21bb:afc2:56e7:4a1b:d4c9
0.08722595208205446


In [11]:
print((phi-epsilon)*(6*l))

106323.9
